In [2]:
import pandas as pd
from pprint import pprint
import sys
import os

REPO_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(REPO_PATH)

from github_api import get_commit_message, get_commit_diff

Mozilla Performance regession dataset

In [ ]:
bugs_with_diff_df = pd.read_json("../datasets/bugs_with_diff.jsonl", lines=True)
bugs_list = bugs_with_diff_df.to_dict(orient='records')

dataset = []

for bug in bugs_list:
    product = bug.get('product')
    component = bug.get('component')
    diff = bug.get('raw_diff')
    response = "0"
    if bug.get('bug_is_perf_regressor'):
        response = "1"

    prompt = f"""[product]:
    {product}
    [component]:
    {component}
    [code diff]:
    {diff}"""
    response = f"{response}"

    instance = {}
    instance['prompt'] = prompt
    instance['response'] = response

    dataset.append(instance)


dataset_df = pd.DataFrame(dataset)
dataset_df.to_json("../datasets/dataset.jsonl", orient="records", lines=True)


ApacheJIT defect prediction dataset for LLMs

In [3]:
apachejit_with_diff_df = pd.read_json("../datasets/jit_dp/apachejit_with_diff.jsonl", lines=True)
apachejit_list = apachejit_with_diff_df.to_dict(orient='records')

sorted_apachejit_list = sorted(apachejit_list, key=lambda x: x['author_date'])

print(sorted_apachejit_list)

new_jit_list = []

OWNER = "apache"

for commit in sorted_apachejit_list:
    project = commit.get('project')
    project_parts = project.split("/")
    repo = project_parts[1]
    commit_hash = commit.get('commit_id')
    message = commit.get('commit_message')
    diff = commit.get('diff')

    if not message or not diff:
        continue

    is_bug_inducing = commit.get('buggy')
    response = "0"
    if is_bug_inducing:
        response = "1"

    prompt = f"""[project]:
    {repo}
    [commit message]:
    {message}
    [code diff]:
    {diff}"""
    response = f"{response}"

    instance = {}
    instance['prompt'] = prompt
    instance['response'] = response

    new_jit_list.append(instance)

new_jit_df = pd.DataFrame(new_jit_list)
new_jit_df.to_json("../datasets/jit_dp/apachejit_llm.jsonl", orient="records", lines=True)


ApacheJIT defect prediction dataset for logistic regression

In [5]:
apachejit_df = pd.read_csv("../datasets/jit_dp/apachejit_train.csv")
apachejit_list = apachejit_df.to_dict(orient='records')

new_jit_list = []

for commit in apachejit_list:
    new_commit = {}

    new_commit['date_created'] = commit.get('author_date')
    new_commit['num_lines_added'] = commit.get('la')
    new_commit['num_lines_deleted'] = commit.get('ld')
    new_commit['num_files_touched'] = commit.get('nf')
    new_commit['num_directories_touched'] = commit.get('nd')
    new_commit['num_subsystems_touched'] = commit.get('ns')
    new_commit['change_entropy'] = commit.get('ent')
    new_commit['num_developers_touched_files'] = commit.get('ndev')
    new_commit['time_from_last_change'] = commit.get('age')
    new_commit['num_changes_in_files'] = commit.get('nuc')
    new_commit['author_experience'] = commit.get('aexp')
    new_commit['author_recent_experience'] = commit.get('arexp')
    new_commit['author_subsystem_experience'] = commit.get('asexp')

    is_bug_inducing = commit.get('buggy')
    
    if is_bug_inducing:
        new_commit['label'] = "1"
    else:
        new_commit['label'] = "0"

    new_jit_list.append(new_commit)

sorted_new_jit_list = sorted(new_jit_list, key=lambda x: x['date_created'])

sorted_new_jit_df = pd.DataFrame(sorted_new_jit_list)
sorted_new_jit_df.replace(["", "null", "N/A", "--"], pd.NA, inplace=True)
sorted_new_jit_clean_df = sorted_new_jit_df.dropna()

sorted_new_jit_clean_df.to_csv("../datasets/jit_dp/apachejit_logreg.csv", index=False)

